<center><h1>POD-Mini: minimized proof of delivery</h1></center>
<center><h2>implemented via Python</h2><center>

# I Intro

In [35]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This is a Python implementaton of Pod-Mini of [zkPod](https://github.com/sec-bit/zkPoD-node) via klefki, for more details, just check the [technial Paper](https://github.com/sec-bit/zkPoD-node).

In [36]:
from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)


G = CG.G

In [37]:
import random

N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
random_f = lambda: CF(random.randint(1, N) % CF.P)

q = random_f()
H = G @ q
    

In [38]:
from hashlib import sha256

hash = lambda x, y: CF(int(sha256(str(x.value + y).encode()).hexdigest(), 16) % N)
hash2 = lambda x, y, z: CF(int(sha256(str(x.value + y + z).encode()).hexdigest(), 16) % N)

# II POD-Mini

### Initializer Phase

In [39]:
data = 123456789
tag = 6

In [40]:
m = CF(data)
o = CF(tag)

In [41]:
sigma = G @ m + H @ o

### Deliver Phase

In [42]:
k_w = random_f()

k = hash(k_w, 1)
k_ = hash(k_w, 2)
k0 = hash(k_w, 3)
k0_ = hash(k_w, 4)

In [43]:
K = G @ k + H @ k_
K0 = G @ k0 + H @ k0_
(K, K0)

(EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::39793097811523310626822321741758299461168550080582449887396657509874940053332, FiniteFieldSecp256k1::72447664277273015751086085629690704559833766979754193437138922518316964557942),
 EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::84664226892148380367510746002696313858638683938660580201254074600825766154004, FiniteFieldSecp256k1::59641691500916952315943681743785424795296112869662704401326844554729021317509))

* $S \rightarrow R$: $(K, K_0)$

In [44]:
c = random_f()

* $R \rightarrow S$: $c$

In [45]:
m_ = k + c * m
o_ = k_ + c * o
z = k0 + c * k
z_ = k0_ + c * k_
(m_, o_, z, z_)

(FiniteFieldCyclicSecp256k1::3266274250312870126755799586930622581175433511683459077496113923446027779925,
 FiniteFieldCyclicSecp256k1::27527682537826277117347448799719704189060150797737792530627481918269770802672,
 FiniteFieldCyclicSecp256k1::44768108737502049977333584307993808698905795347533991203489299605766363364588,
 FiniteFieldCyclicSecp256k1::13226651750639271385124501248413674651400493703103141148991529253317127530053)

* $S \rightarrow R: (\bar{m}, \bar{o}, z, z')$

R should verify that:

$$
Com(m; o)^c \cdot Com(k_0;k'_0) \stackrel{?}{=} Com(\bar{m}; \bar{o})\\
Com(k_0;k'_0) \cdot Com(k;k')^c \stackrel{?}{=} Com(z;z')
$$

In [46]:
assert sigma @ c + K == G @ m_ + H @ o_

In [47]:
assert K0 + (K @ c) == G @ z + H @ z_

### Reveal-phase

* $R \rightarrow J: \rho$
* $S \rightarrow J: k_{\omega}$

$$
z \stackrel{?}{=} H(k_{\omega}, 3) + c \cdot H(k_{\omega}, 1)
$$

In [48]:
assert z == hash(k_w, 3) + c @ hash(k_w, 1)

In [49]:
assert m == (m_ - hash(k_w, 1)) / c

In [50]:
(m_ - hash(k_w, 1)) / c

FiniteFieldCyclicSecp256k1::123456789

# POD-AS

In [51]:
from IPython.display import Image

In [52]:
f = Image("lena-mini.jpg")

### Init Phase

In the init-phase, the data file is splitted into a block matrix of $n × s$ . Each row of the matrix is called a $block$, which consists of $s$ slices. The initializer adds one additional column of random slices $m_{0i}$ to the matrix for padding. The slices of $m_{0i}$ are used for blind factors as o in PoD-Mini.

In [53]:
import numpy as np

In [54]:
Image(f.data)


In [55]:
M = np.array(list(map(CF, f.data))).reshape(-1, 5)

In [56]:
w, h = M.shape

In [57]:
Pad = np.matrix([random_f() for _ in range(0, w)])

In [58]:
Pm = np.concatenate((Pad.T, M), axis=1).tolist()

In [59]:
n, s = np.matrix(Pm).shape

The initializer needs to generate $s + 1$ group elements randomly.

In [60]:
U = [G @ random_f() for _ in range(0, s + 1)]

In [61]:
from functools import reduce 

def v_multi(a: [CF], g: [ECG]) -> [ECG]:
    return reduce(lambda x,y: x+y,
                  list(map(lambda a: a[0] @ a[1], zip(g, a))))


In [62]:
delta = [U[0]@M[0][i] + v_multi(M[1:][i], U[1:]) for i in range(1, s-1)]

### Deliver Phase

In [63]:
kw = random_f()

In [64]:
k = [[hash2(kw, i, j) for j in range(0, s+1)] for i in range(0, n+1)]

In [65]:
np.matrix(k).shape

(206, 7)

In [66]:
np.matrix(Pm).shape

(205, 6)

Then $S$ constructs commitments $K$ i to $i$-th row of keys, including the leftmost key $k_{i0}$ on each row.

In [67]:
from multiprocessing import Pool
def mapper(i): return U[0] @ k[i][0] + v_multi(k[i], U[1:])
p = Pool(4)


In [68]:
K = list(map(mapper, range(0, n)))

$S \rightarrow R: \mathbf{K}_[0, n]$